# Семинар по рекуррентным нейронным сетям
На этом семинаре мы обучим несколько рекуррентных архитектур для решения задачи сентимент-анализа, то есть предсказания метки тональности предложения.

В общем случае рекуррентная нейронная сеть предназначена для обработки последовательности произвольной длины. Однако при реализации метода оказывается проще зафиксировать длину последовательности (даже в pytorch с их динамическими графами :) Поэтому мы пока поступим так, но вернемся к этому вопросу ниже.

Сначала мы разберемся с RNN в pytorch, а затем сами реализуем наиболее популярную архитектуру.

Задание сделано так, чтобы его можно было выполнять на CPU, однако RNN - это ресурсоемкая вещь, поэтому на GPU с ними работать приятнее. Можете попробовать использовать [https://colab.research.google.com](https://colab.research.google.com) - бесплатное облако с GPU.

### Гиперпараметры

In [1]:
vocab_size = 20000 
index_from = 3
n_hidden = 128
n_emb = 300
seq_len = 32

batch_size = 128
learning_rate = 0.001
num_epochs = 50

use_gpu = True

### Загрузка данных
Функция load_matrix_imdb скачивает матричные данные, перемешивает и загружает их в numpy-массивы.

Если у вас не установлен wget, скачайте [архив imdb.npz](https://s3.amazonaws.com/text-datasets/imdb.npz)

In [2]:
from rnn_utils import load_matrix_imdb
import numpy as np
import torch
import torch.utils.data

In [3]:
np.random.seed(0)
(X_train, y_train), (X_test, y_test) = load_matrix_imdb(num_words=vocab_size,
                                                        maxlen=seq_len)

In [4]:
set(y_train) # binary classification

{0, 1}

In [5]:
X_train.shape, X_test.shape

((25000, 32), (25000, 32))

In [6]:
X_train[0] # sequence of coded words

array([1.000e+00, 1.400e+01, 2.200e+01, 1.600e+01, 4.300e+01, 5.300e+02,
       9.730e+02, 1.622e+03, 1.385e+03, 6.500e+01, 4.580e+02, 4.468e+03,
       6.600e+01, 3.941e+03, 4.000e+00, 1.730e+02, 3.600e+01, 2.560e+02,
       5.000e+00, 2.500e+01, 1.000e+02, 4.300e+01, 8.380e+02, 1.120e+02,
       5.000e+01, 6.700e+02, 2.000e+00, 9.000e+00, 3.500e+01, 4.800e+02,
       2.840e+02, 5.000e+00])

In [7]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [8]:
train_dset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.long), 
                               torch.tensor(y_train, dtype=torch.long))

In [9]:
test_dset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long), 
                               torch.tensor(y_test, dtype=torch.long))

In [10]:
train_loader = torch.utils.data.DataLoader(train_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4
                         )

In [11]:
test_loader = torch.utils.data.DataLoader(test_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4
                         )

### Сборка и обучение RNN в pytorch

In [12]:
import os
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable

Наша нейросеть будет обрабатывать входную последовательность по словам (word level). Мы будем использовать простую и стандарную рекуррентную архитектуру для сентимент-анализа: слой представлений, слой LSTM и полносвязный слой, предсказывающий выход по последнему скрытому состоянию.

Ниже даны шаблоны реализации нейросети и ее обучения. Допишите класс и функции обучения так, чтобы класс реализовывал описанную архитектуру, а вызов функции train не выдавал ошибок :)

In [13]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, \
                 batch_size, use_gpu, rec_layer):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.use_gpu = use_gpu

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = rec_layer(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
    
    def forward(self, sentences):
        embedding = self.word_embeddings(sentences)
        out, hidden = self.rnn(embedding)
        res = self.hidden2label(out[-1])
        return nn.functional.sigmoid(res)

In [14]:
model = LSTMClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       use_gpu=use_gpu,
                       rec_layer = nn.RNN)
if use_gpu:
    model = model.cuda()

RuntimeError: torch.cuda.FloatTensor is not enabled.

[Исходный код LSTM](http://pytorch.org/docs/master/_modules/torch/nn/modules/rnn.html#LSTM)

In [79]:
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(size_average=False)

In [80]:
def train_epoch(train_loader, model, lossfun, optimizer, use_gpu):
    model.train()
    for it, traindata in enumerate(train_loader):
        train_inputs, train_labels = traindata
        train_labels = torch.squeeze(train_labels)

        if use_gpu:
            train_inputs = train_inputs.to(torch.device('cuda'))
            train_labels = train_labels.to(torch.device('cuda'))

        model.zero_grad()        
        output = model(train_inputs.t())

        loss = lossfun(output.view(-1), train_labels.float())
        loss.backward()
        optimizer.step()

def evaluate(loader, model, lossfun, use_gpu):
    model.evaluate()
    total_acc = 0.0
    total_loss = 0.0
    total = 0.0
    for it, data in enumerate(loader):
        inputs, labels = data
        labels = torch.squeeze(labels)
        
        if use_gpu:
            inputs = inputs.to(torch.device('cuda'))
            labels = labels.to(torch.device('cuda'))

        ### your code here

        output = model(inputs.t())
        loss = lossfun(output.view(-1), labels.float())
        total_loss += loss.item()

        # calc testing acc        
        pred = output.view(-1) > 0.5
        correct = pred == labels.byte()
        total_acc += torch.sum(correct).item() / len(correct)

    total = it + 1
    return total_loss / total, total_acc / total
    

def train(train_loader, test_loader, model, lossfun, optimizer, \
          use_gpu, num_epochs):
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []
    for epoch in range(num_epochs):
        train_epoch(train_loader, model, lossfun, optimizer, use_gpu)
        train_loss, train_acc = evaluate(train_loader, model, lossfun, use_gpu)
        train_loss_.append(train_loss)
        train_acc_.append(train_acc)
        test_loss, test_acc = evaluate(test_loader, model, lossfun, use_gpu)
        test_loss_.append(test_loss)
        test_acc_.append(test_acc)

        print(f'Epoch: {epoch+1:3d}/{num_epochs:3d} '
              f'Training Loss: {train_loss_[epoch]:.3f}, Testing Loss: {test_loss_[epoch]:.3f}, '
              f'Training Acc: {train_acc_[epoch]:.3f}, Testing Acc: {test_acc_[epoch]:.3f}')

    return train_loss_, train_acc_, test_loss_, test_acc_

In [81]:
a, b, c, d = train(train_loader, test_loader, model, lossfun, optimizer, use_gpu, 15)

Epoch:   1/ 15 Training Loss: 5.007, Testing Loss: 223.259, Training Acc: 0.986, Testing Acc: 0.702
Epoch:   2/ 15 Training Loss: 0.314, Testing Loss: 222.826, Training Acc: 1.000, Testing Acc: 0.708
Epoch:   3/ 15 Training Loss: 0.562, Testing Loss: 228.732, Training Acc: 0.999, Testing Acc: 0.707
Epoch:   4/ 15 Training Loss: 0.802, Testing Loss: 238.845, Training Acc: 0.998, Testing Acc: 0.703
Epoch:   5/ 15 Training Loss: 0.648, Testing Loss: 274.602, Training Acc: 0.999, Testing Acc: 0.708
Epoch:   6/ 15 Training Loss: 0.201, Testing Loss: 287.141, Training Acc: 1.000, Testing Acc: 0.711
Epoch:   7/ 15 Training Loss: 0.189, Testing Loss: 265.573, Training Acc: 1.000, Testing Acc: 0.705
Epoch:   8/ 15 Training Loss: 0.148, Testing Loss: 305.877, Training Acc: 1.000, Testing Acc: 0.705
Epoch:   9/ 15 Training Loss: 0.539, Testing Loss: 260.845, Training Acc: 0.999, Testing Acc: 0.704
Epoch:  10/ 15 Training Loss: 0.220, Testing Loss: 270.488, Training Acc: 1.000, Testing Acc: 0.705


Нерегуляризованные LSTM часто быстро переобучаются. Чтобы с этим бороться, часто используют L2-регуляризацию и дропаут.
Однако способов накладывать дропаут на рекуррентный слой достаточно много, и далеко не все хорошо работают. Мы реализуем дропаут, описанный в [статье Гала и Гарамани](https://arxiv.org/abs/1512.05287).
Для этого нам потребуется самостоятельно реализовать LSTM.

### Самостоятельная реализация LSTM

Для начала реализуйте LSTM, не обращая внимание на параметр dropout, и протестируйте модель. На каждом шаге ячейка LSTM обновляет скрытое состояние и память по следующим формулам:
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t) \nonumber
$$

In [77]:
from torch.nn.parameter import Parameter

In [78]:
Parameter?

In [ ]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0):
        super(MyLSTM, self).__init__()
        ### your code here
        
        
    def reset_params(self):
        """
        initialization as in Pytorch
        do not forget to call this method!
        """
        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
            
    def forward(self, input, hidden):
        ### your code here
        

In [ ]:
model = LSTMClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       use_gpu=use_gpu,
                       rec_layer=MyLSTM)
if use_gpu:
    model = model.cuda()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.BCELoss(size_average=False)
a, b, c, d = train(train_loader, test_loader, model, lossfun, optimizer, \
      use_gpu, 15)

Теперь реализуйте дропаут для рекуррентного слоя. Как и в сетях прямого распространения, дропаут можно накладывать на вход и скрытое состояние ($x_t$ и $h_t$). Ключевая идея дропаута Гала состоит в том, что бинарная маска должна быть одинаковая для всех моментов времени (но своя для разных объектов). Кроме того, статья утверждает, что одновременно с бинарным дропаутом нужно использовать L$_2$-регуляризацию. Ее тоже можно включить (параметр weight_decay в оптимизаторе).

Формулы ячейки LSTM с бинарным дропаутом ($b_x$ и $b_h$ - бинарные маски):

$$
i = \sigma((h_{t-1} \odot b_h) W^i + (x_t \odot b_x) U^i+b_i) \quad
o = \sigma((h_{t-1} \odot b_h)W^o + (x_t \odot b_x) U^o+b_o) 
$$
$$
f = \sigma((h_{t-1} \odot b_h)W^f + (x_t \odot b_x) U^f+b_f) \quad 
g = tanh((h_{t-1} \odot b_h) W^g + (x_t \odot b_x) U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t)
$$

In [ ]:
### your code here

Протестируйте полученную модель, сравните итоговое качество на тестовой выборке с нерегуляризованной моделью.

In [ ]:
### your code here

### Бонусные задания
Вы можете выполнить одно или несколько бонусных (необязательных) заданий на исследование рекуррентной нейросети.

1. __Инициализация (2 балла).__ В разных фреймворках по-разному инициализируют веса рекуррентного слоя и эмбеддингов, а также сдвиги. Например, в pytorch эмбеддинги инициализируются из нормального распределения, а веса рекуррентного слоя - из равномерного (см. [исходники 1](http://pytorch.org/docs/master/_modules/torch/nn/modules/rnn.html#LSTM) и [исходники 2](http://pytorch.org/docs/master/_modules/torch/nn/modules/sparse.html#Embedding)). Рассмотрите следующие варианты инициализации (каждый пункт означает, что поменять в исходной инициализации pytorch):
    * Инициализация эмбеддингов из равномерного распределения [-0.05, 0.05] (стандартная практика)
    * Инициализация весов hidden-to-hidden ортогональной матрицей (см. [реализацию в theano](https://github.com/Lasagne/Lasagne/blob/master/lasagne/init.py#L327-L367))
    * Инициализация весов рекуррентного слоя из нормального распределения (как в embedding)
    * Инициализация сдвига forget gate единицей (чтобы "начинать с запоминания")
    Сравните качество работы нерегуляризованной LSTM с такими инициализациями. Можно ли сказать, что в pytorch грамотно выбрана инициализация по умолчанию?
    
1. __Начальное состояние (1 балл).__ В наших экспериментах мы всегда инициализировали начальное состояние (h_0 и c_0) нулями. Попробуйте обучать эти векторы. Повысится ли качество? Впрочем, универсального рецепта тут нет, это тоже своеобразный гиперпараметр :)

1. __Переменная длина (2 балла).__ Сравните качество работы модели с seq_len = 50, 200, 400 и переменной длиной. Чтобы реализовать поддержку последовательностей переменной длины, обычно используют паддинг (можно начать гуглить [отсюда](https://discuss.pytorch.org/t/how-to-handle-variable-length-inputs-sentences/5407)).

1. __Визуализация (до 3-х баллов).__ Попробуйте сделать наглядную визуализацию изменений состояний в рекуррентной нейросети. Это задание больше творческое, чем с конкретными рекомендациями. Можно попробовать показывать, на каких входных словах меняется скрытое состояние и как это влияет на выход нейросети. Поскольку на семинаре мы загружали матричные данные, вам придется также разобраться с загрузкой полноценных текстовых данных. С этим поможет модуль torchtext (в нем есть готовые загрузчики IMDB). 

1. __Предобученные векторы представлений (1 балл).__ В моделях word level слой представлений, как правило, имеет очень большое число параметров. Чтобы не обучать их с нуля, часто используют предобученные векторы (и дообучают их). Обычно это позволяет поднять качество на несколько процентов. Сравните качесто такой модели и инициализированной случайно. С загрузкой векторов GloVe или Word2Vec может помочь модуль torchtext.